### Importings

In [29]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

# set your path
DATASET_PATH = '../datasets/'

In [30]:
df = pd.read_sas(DATASET_PATH + '/ess6e02_4.sas7bdat', encoding = 'latin-1')

### Filter columns

In [31]:
cols = ['cntry', 'sclmeet', 'inprdsc', 'aesfdrk', 'health', 'hlthhmp', 'sclact', 'crmvct',
        'fltsd', 'fltdpr', 'fltanx', 'flteeff', 'slprl', 'cldgng', 'fltlnl',
        'tmimdng', 'tmabdng', 'tmendng', 'flapppl', 'lchshcp',
        'stflife', 'happy', 'enjlf', 'wrhpp', 'fltpcfl', 'enrglot',
        'dclvlf', 'accdng', 'dngval', 'optftr', 'lotsgot', 'pstvms', 
        'flclpla', 'pplahlp', 'rehlppl', 'trstprl', 'trstlgl', 'trstplc', 'trstplt', 'trstprt', 'trstep', 'trstun', 
        'ppltrst', 'pplfair', 'pplhlp','deaimpp', 'flrms', 'sedirlf', 'wrbknrm']

df = df[cols]

### Dataprep

In [32]:
df['cntry'] = df['cntry'].map({'AL':'Albania','BE':'Belgium','BG':'Bulgaria','CH':'Switzerland','CY':'Cyprus','CZ':'Czechia','DE':'Germany','DK':'Denmark','EE':'Estonia','ES':'Spain','FI':'Finland','FR':'France','GB':'United Kingdom','HU':'Hungary','IE':'Ireland','IL':'Israel','IS':'Iceland','IT':'Italy','LT':'Lithuania','NL':'Netherlands','NO':'Norway','PL':'Poland','PT':'Portugal','RU':'Russian Federation','SE':'Sweden','SI':'Slovenia','SK':'Slovakia','UA':'Ukraine','XK':'Kosovo'})

In [33]:
df['sclmeet'] = df['sclmeet'].map({1:'Never',2:'Less than once a month',3:'Once a month',4:'Several times a month',5:'Once a week',6:'Several times a week',7:'Every day',77:'Refusal',88:'Dont know',99:'No answer'})
df['inprdsc'] = df['inprdsc'].map({0:'None',1:'1',2:'2',3:'3',4:'4-6',5:'7-9',6:'10 or more',77:'Refusal',88:'Dont know',99:'No answer'})
df['aesfdrk'] = df['aesfdrk'].map({1:'Very safe',2:'Safe',3:'Unsafe',4:'Very unsafe',7:'Refusal',8:'Dont know',9:'No answer'})
df['health'] = df['health'].map({1:'Very good',2:'Good',3:'Fair',4:'Bad',5:'Very bad',7:'Refusal',8:'Dont know',9:'No answer'})
df['hlthhmp'] = df['hlthhmp'].map({1:'Yes a lot',2:'Yes to some extent',3:'No',7:'Refusal',8:'Dont know',9:'No answer'})

In [34]:
for col in ['sclact', 'crmvct']:
    df[col] = df[col].map({1:'Much less than most',2:'Less than most',3:'About the same',
                           4:'More than most',5:'Much more than most',7:'Refusal',8:'Dont know',9:'No answer'})

In [35]:
for col in ['fltsd', 'fltdpr', 'fltanx', 'flteeff', 'slprl', 'cldgng', 'fltlnl',
            'enjlf', 'wrhpp', 'fltpcfl', 'enrglot']:
    df[col] = df[col].map({1:'None or almost none of the time',2:'Some of the time',3:'Most of the time',
                           4:'All or almost all of the time',7:'Refusal',8:'Dont know',9:'No answer'})

In [36]:
for col in ['dclvlf', 'accdng', 'dngval', 'optftr', 'lotsgot', 'pstvms', 'flclpla', 'lchshcp', 'flrms', 'wrbknrm']:
    df[col] = df[col].map({1:'Agree strongly',2:'Agree',3:'Neither agree nor disagree',4:'Disagree',
                           5:'Disagree strongly',7:'Refusal',8:'Dont know',9:'No answer'})

In [37]:
df.head(3)

,cntry,sclmeet,inprdsc,aesfdrk,health,hlthhmp,sclact,crmvct,fltsd,fltdpr,fltanx,flteeff,slprl,cldgng,fltlnl,tmimdng,tmabdng,tmendng,flapppl,lchshcp,stflife,happy,enjlf,wrhpp,fltpcfl,enrglot,dclvlf,accdng,dngval,optftr,lotsgot,pstvms,flclpla,pplahlp,rehlppl,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,ppltrst,pplfair,pplhlp,deaimpp,flrms,sedirlf,wrbknrm
0,Albania,Once a week,3,Very safe,Fair,No,About the same,Less than most,Some of the time,Most of the time,Some of the time,Some of the time,All or almost all of the time,NaN,None or almost none of the time,8.0,8.0,8.0,10.0,Agree,6.0,8.0,Some of the time,Most of the time,Most of the time,Most of the time,Agree strongly,Agree,Agree,Agree strongly,Disagree,Agree,Agree,1.0,6.0,0.0,0.0,2.0,0.0,0.0,2.0,2.0,5.0,3.0,0.0,5.0,Disagree,3.0,Disagree
1,Albania,Once a month,1,Unsafe,Good,No,Less than most,Less than most,All or almost all of the time,All or almost all of the time,All or almost all of the time,Some of the time,Most of the time,NaN,All or almost all of the time,10.0,5.0,0.0,10.0,Agree strongly,0.0,0.0,None or almost none of the time,None or almost none of the time,Some of the time,None or almost none of the time,Agree strongly,Agree,Agree strongly,Agree,Agree strongly,Neither agree nor disagree,Neither agree nor disagree,0.0,6.0,0.0,0.0,10.0,8.0,10.0,10.0,10.0,0.0,88.0,0.0,0.0,Agree,10.0,Agree
2,Albania,Every day,10 or more,Unsafe,Very good,No,About the same,Much less than most,Some of the time,Some of the time,None or almost none of the time,Most of the time,Some of the time,NaN,Some of the time,10.0,10.0,10.0,3.0,Agree,5.0,5.0,Most of the time,Some of the time,Some of the time,Most of the time,Agree strongly,Agree,Agree strongly,Agree strongly,Agree strongly,Agree,Disagree,0.0,6.0,88.0,2.0,0.0,0.0,0.0,8.0,5.0,5.0,5.0,4.0,5.0,Disagree strongly,10.0,Disagree strongly


### Exporting

In [38]:
df.to_csv(DATASET_PATH + 'df_tableau.csv', index=False)